# CS 345 Project
#### _Ben Spencer and Matteo Selvaggio_

#### **Part 0: Importing Data and Libraries**

In [ ]:
# installing zstandard package to unpack .zst compressed files
# %pip install zstandard
import zstandard as zstd

# installing chess package which contains useful methods for processing chess boards.
# %pip install chess
import chess

# importing numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# importing sklearn methods
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# importing keras with GPU
# If using Anaconda, use the following commands in a local conda terminal:
# %pip install --upgrade pip
# %conda create -n tf tensorflow
# %conda activate tf
# %pip install tensorflow[and-cuda]
import tensorflow as tf